In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Flatten, SimpleRNN, LSTM
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
import time

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv')
df.head(10)

def getTrainTest(url, splitPercent=0.8):
    df = pd.read_csv(url, usecols=[1], engine='python')
    print(df)
    data = np.array(df.values.astype('float32'))
    print(data)
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n * splitPercent)
    trainData = data[range(split)]
    testData = data[split:]
    return trainData, testData, data

In [9]:
sunspotsData = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
trainData, testData, data = getTrainTest(sunspotsData)

def createNN(input_shape, activation):
    model = Sequential()
    
    model.add(Dense(
        64,
        input_shape=input_shape,
        activation=activation[0]
    ))

    model.add(Flatten())
    
    model.add(Dense(
        32,
        input_shape=input_shape,
        activation=activation[0]
    ))
    
    model.add(Flatten())
    
    model.add(Dense(
        1,
        input_shape=input_shape,
        activation=activation[0]
    ))
   
    model.add(Flatten())
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def createRNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()

    model.add(SimpleRNN(
        hidden_units, 
        input_shape=input_shape, 
        activation=activation[0]
    ))

    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def createLSTM(hidden_units, dense_units, input_shape, activation):
    model = Sequential()

    model.add(LSTM(
        hidden_units, 
        input_shape=input_shape, 
        activation=activation[0]
    ))

    model.add(Dense(units=dense_units, activation=activation[1]))

    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

      Sunspots
0         58.0
1         62.6
2         70.0
3         55.7
4         85.0
...        ...
2815      71.8
2816      50.3
2817      55.8
2818      33.3
2819      33.4

[2820 rows x 1 columns]
[[58. ]
 [62.6]
 [70. ]
 ...
 [55.8]
 [33.3]
 [33.4]]


In [10]:
def getXY(dat, timeSteps):
    # Indices of target array
    indY = np.arange(timeSteps, len(dat), timeSteps)
    Y = dat[indY]

    # Prepare X
    rowsX = len(Y)
    X = dat[range(timeSteps * rowsX)]
    X = np.reshape(X, (rowsX, timeSteps, 1))   

    return X, Y

def printError(trainY, testY, trainPredict, testPredict):    
    # Error of predictions
    trainRMSE = math.sqrt(mean_squared_error(trainY, trainPredict))
    testRMSE = math.sqrt(mean_squared_error(testY, testPredict))
    trainR2 = r2_score(trainY, trainPredict)
    testR2 = r2_score(testY, testPredict)

    print('Train RMSE: %.3f RMSE' % (trainRMSE))
    print('Test RMSE: %.3f RMSE' % (testRMSE))   
    print('Train R2: %.3f R2' % (trainR2))
    print('Test R2: %.3f R2' % (testR2))
 
timeSteps = 12
trainX, trainY = getXY(trainData, timeSteps)
testX, testY = getXY(testData, timeSteps)

In [11]:
modelRNN = createRNN(hidden_units=3, dense_units=1, input_shape=(timeSteps,1), activation=['tanh', 'tanh'])
modelNN = createNN(input_shape=(timeSteps,1), activation=['relu', 'relu'])
modelLSTM = createLSTM(hidden_units=3, dense_units=1, input_shape=(timeSteps,1), activation=['tanh', 'tanh'])

timeStartRNN = time.time()
modelRNN.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)
timeEndRNN = time.time()

timeStartNN = time.time()
modelNN.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)
timeEndNN = time.time()

timeStartLSTM = time.time()
modelLSTM.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)
timeEndLSTM = time.time()

Epoch 1/20
187/187 - 8s - loss: 0.0939 - 8s/epoch - 45ms/step
Epoch 2/20
187/187 - 8s - loss: 0.0601 - 8s/epoch - 43ms/step
Epoch 3/20
187/187 - 8s - loss: 0.0462 - 8s/epoch - 43ms/step
Epoch 4/20
187/187 - 8s - loss: 0.0386 - 8s/epoch - 43ms/step
Epoch 5/20
187/187 - 8s - loss: 0.0335 - 8s/epoch - 43ms/step
Epoch 6/20
187/187 - 8s - loss: 0.0287 - 8s/epoch - 43ms/step
Epoch 7/20
187/187 - 8s - loss: 0.0251 - 8s/epoch - 44ms/step
Epoch 8/20
187/187 - 8s - loss: 0.0218 - 8s/epoch - 44ms/step
Epoch 9/20
187/187 - 8s - loss: 0.0191 - 8s/epoch - 44ms/step
Epoch 10/20
187/187 - 8s - loss: 0.0170 - 8s/epoch - 43ms/step
Epoch 11/20
187/187 - 8s - loss: 0.0151 - 8s/epoch - 43ms/step
Epoch 12/20
187/187 - 8s - loss: 0.0138 - 8s/epoch - 43ms/step
Epoch 13/20
187/187 - 8s - loss: 0.0122 - 8s/epoch - 44ms/step
Epoch 14/20
187/187 - 8s - loss: 0.0116 - 8s/epoch - 44ms/step
Epoch 15/20
187/187 - 8s - loss: 0.0107 - 8s/epoch - 44ms/step
Epoch 16/20
187/187 - 8s - loss: 0.0097 - 8s/epoch - 44ms/step
E

In [12]:
trainPredictNN = modelNN.predict(trainX)
testPredictNN = modelNN.predict(testX)

trainPredictRNN = modelRNN.predict(trainX)
testPredictRNN = modelRNN.predict(testX)

trainPredictLSTM = modelLSTM.predict(trainX)
testPredictLSTM = modelLSTM.predict(testX)

print('\nNN')
printError(trainY, testY, trainPredictNN, testPredictNN)
print('Time: ', timeEndNN - timeStartNN)

print('\nRNN')
printError(trainY, testY, trainPredictRNN, testPredictRNN)
print('Time: ', timeEndRNN - timeStartRNN)

print('\nLSTM')
printError(trainY, testY, trainPredictLSTM, testPredictLSTM)
print('Time: ', timeEndLSTM - timeStartLSTM)

2/2 [==============================] - 0s 13ms/step

NN
Train RMSE: 0.048 RMSE
Test RMSE: 0.074 RMSE
Train R2: 0.892 R2
Test R2: 0.880 R2
Time:  13.962080955505371

RNN
Train RMSE: 0.079 RMSE
Test RMSE: 0.120 RMSE
Train R2: 0.704 R2
Test R2: 0.690 R2
Time:  162.77355217933655

LSTM
Train RMSE: 0.061 RMSE
Test RMSE: 0.088 RMSE
Train R2: 0.822 R2
Test R2: 0.833 R2
Time:  23.13651204109192


### Resultados de NN
Valor RMSE de prueba: 0.074
Valor R cuadrado de prueba: 0.88
El 88% de los cambios de las variables dependientes se explican por los cambios de las variables independientes.

Pros de NN:
- Es la estructura que obtiene los mejores resultados.
- Menor tiempo de entrenamiento.

Contras de NN:
- Puede presentar un poco de overfitting.
- Requiere de un mayor número de capas ocultas para obtener buenos resultados.

### Resultados de RNN
Valor RMSE de prueba: 0.12
Valor R cuadrado de prueba: 0.69
El 84.7% de los cambios de las variables dependientes se explican por los cambios de las variables independientes.

Pros de RNN:
- Obtiene resultados casi tan buenos como los de NN.
- Requiere de un menor número de capas ocultas para obtener buenos resultados.

Contras de RNN:
- Puede presentar un poco de overfitting.
- Tiempo de entrenamiento mayor que el de los otros modelos.

### Resultados de LSTM
Valor RMSE de prueba: 0.088
Valor R cuadrado de prueba: 0.833
El 82.1% de los cambios de las variables dependientes se explican por los cambios de las variables independientes.

Pros de LSTM:
- Requiere de un menor número de capas ocultas.
- Tiempo de entrenamiento menor que el de RNN.

Contras de LSTM:
- Obtiene resultados peores que los de NN y RNN.
- Puede presentar un poco de overfitting.

### Estructura óptima
Según los resultados obtenidos, la estructura Feed Forward Neural Network es la óptima para este problema, con un valor R cuadrado de 0.88 y un RMSE de 0.074 en los datos de prueba. Al comparar con la teória, se puede concluir que la estructura de la red neuronal recurrente no es la más adecuada para este tipo de problemas, ya que está diseñada para trabajar con datos secuenciales, como series de tiempo. En este caso, los datos no son secuenciales, por lo que la red neuronal recurrente no es la más adecuada para este problema. De similar manera, la estructura LSTM tampoco es la adecuada para este tipo de problemas, ya que también está diseñada para trabajar con datos secuenciales. Adicionalmente, la red neuronal LSTM está diseñada para trabajar con datos secuenciales que tienen una dependencia a largo plazo. En otras palabras, está diseñada para trabajar con datos en los cuales los datos de entrada tienen una dependencia con los datos de entrada de hace mucho tiempo. En este caso, los datos no tienen una dependencia a largo plazo, por lo que la red neuronal LSTM no es la más adecuada para este problema. Finalmente, la estructura Feed Forward Neural Network es la más adecuada para este problema, ya que los datos no son secuenciales y no tienen una dependencia a largo plazo.